## 公众号: 大邓和他的Python

[![](img/management_data_mining_with_python_course2.png)](https://hidadeng.github.io/blog/management_python_course/)


BERTopic | 使用推特数据构建动态话题模型


在本文中将使用BERTopic库，对美国前总统Trump推特数据集，构建动态主题模型DTM(Dynamic Topic Modeling)，可视化文档数据集中不同主题随时间的演变(变迁)。

<br>


<br>

## 安装

为保证代码可复现，保证你我电脑中bertopic版本一致，先查看大邓电脑的bertopic版本

In [32]:
import bertopic

#本文bertopic版本
bertopic.__version__

'0.12.0'

In [3]:
#推荐指定版本安装；
#!pip3 install bertopic==0.12.0

#不指定版本安装
!pip3 install bertopic

<br>

## 导入数据
这里准备了twitter账号 @realDonalTrump 中 2021年的推特数据，  数据存储于 **trump_twitter_2021.csv**。

- 我们只分析原推特，不分析每条推特的回复。
- 因为要分析推特随时间的主题变化，需要准备 **推特** 及对应的 **推文时间**

In [51]:
import re
import pandas as pd
from datetime import datetime

# 导入数据
trump_df = pd.read_csv('trump_twitter_2021.csv')
trump_df.head()

id                                               text  \
0    98454970654916608  Republicans and Democrats have both created ou...   
1  1234653427789070336  I was thrilled to be back in the Great city of...   
2  1218010753434820614  RT @CBS_Herridge: READ: Letter to surveillance...   
3  1304875170860015617  The Unsolicited Mail In Ballot Scam is a major...   
4  1218159531554897920  RT @MZHemingway: Very friendly telling of even...   

  isRetweet isDeleted              device  favorites  retweets  \
0         f         f           TweetDeck         49       255   
1         f         f  Twitter for iPhone      73748     17404   
2         t         f  Twitter for iPhone          0      7396   
3         f         f  Twitter for iPhone      80527     23502   
4         t         f  Twitter for iPhone          0      9081   

                  date isFlagged  
0  2011-08-02 18:07:48         f  
1  2020-03-03 01:34:50         f  
2  2020-01-17 03:22:47         f  
3  2020-09-12 20:10:58         f  
4  2020-01-17 13:13:59         f

<br>

## 预处理
- 使用正则表达式 清除推文中的http链接
- 剔除@符
- 使用正则表达式 剔除 非英文字符

In [34]:
import re

#预处理函数clean_text
def clean_text(text):
    text = re.sub("http\S+", "", text).lower()
    text = " ".join([w for w in text.split() if w[0]!='@'])
    text = re.sub("[^a-zA-Z]+", " ", text).lower()
    return text


test_text = 'hello @Apple, https://apple.com 李John'
#验证函数有效性
clean_text(text=test_text)

'hello john'

<br>

- 对text字段使用预处理函数clean_text
- 只保留原推文
- 准备推特tweets和时间戳timestamps

<br>

In [22]:
#清洗字段text
trump_df['text'] = trump_df['text'].apply(clean_text)

#只保留特朗普原推文(剔除特朗普的Retweet)
#推文内容不能为”“
trump_df = trump_df.loc[(trump_df['isRetweet'] == "f") & (trump_df['text'] != ""), :]

#准备tweets及对应的timestamps
tweets = trump_df['text'].to_list()
timestamps = trump_df['date'].to_list()

tweets[0]

'republicans and democrats have both created our economic problems '


<br>

## 初始化BERTopic
在模型初始化阶段，使用所有推文数据， 会忽略时间维度。 该步骤会把所有时间段中出现的主题都提前训练识别出来。

In [25]:
from bertopic import BERTopic

#大邓这里，运行了不到1小时
#特朗普比较活跃，什么内容都会参与，所以这里设置一个话题数下限为35，话题数上限不设置
topic_model = BERTopic(min_topic_size=35, verbose=True)
topics, _ = topic_model.fit_transform(tweets)

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Batches:   0%|          | 0/1418 [00:00<?, ?it/s]

2022-12-04 22:04:02,964 - BERTopic - Transformed documents to Embeddings
2022-12-04 22:05:13,606 - BERTopic - Reduced dimensionality
2022-12-04 22:05:17,814 - BERTopic - Clustered reduced embeddings


<br>
抽取出所有的话题

In [46]:
freq = topic_model.get_topic_info()

#话题总数
print(len(freq))
freq.head(10)

169


Topic  Count                                   Name
0     -1  15098                        -1_the_to_is_of
1      0   3182           0_run_president_trump_donald
2      1   1821            1_crowd_carolina_join_thank
3      2   1084           2_golf_course_doral_scotland
4      3   1030       3_border_wall_immigration_mexico
5      4    811          4_china_trade_tariffs_chinese
6      5    642  5_obamacare_healthcare_repeal_website
7      6    638          6_hillary_clinton_crooked_she
8      7    607                        7_amp_it_you_to
9      8    562              8_media_fake_news_failing

<br>

-1 意识是所有的离群点(异类)推文，应该被忽略掉。接下来让我们看一下Topic-4的特征词及其权重

In [ ]:
#topic-4的特征词及权重
topic_model.get_topic(4)

[('china', 0.05289416225000891),
 ('tariffs', 0.024471004754487165),
 ('trade', 0.02437576425026641),
 ('chinese', 0.013643270667358017),
 ('us', 0.011206804363719649),
 ('farmers', 0.01113584970813823),
 ('our', 0.010197907480148342),
 ('deal', 0.010014612658730073),
 ('we', 0.009043537683534882),
 ('countries', 0.00901653033214627)]

<br>

在二维空间中使用 Intertopic Distance Map可视化所有主题。该图可以让我们继续创建DTM前，判断主题数设置的是否充分够用。

In [50]:
fig = topic_model.visualize_topics()
fig

<br>

## 构建DTM
在 构建动态主题模型 前， 不同时间段中出现的主题需要预先都训练好。

- docs 文档数据，对应于本文的tweets
- timestamps 时间戳，对应于本文的timestamps
- global_tuning 是否将某个主题在 时间t 的主题表示向量 与 其全局主题表示向量 进行平均
- evolution_tuning 是否将某个主题在 时间t 的主题表示向量 与 该主题在时间t-1 的主题表示向量 进行平均
- nr_bins 时间段内含有的时间戳(点)数量。在数千个不同的时间戳中提取主题在计算上是低效的, 可以合并20个时间戳为一个时间段

In [37]:
topics_over_time = topic_model.topics_over_time(docs=tweets, 
                                                timestamps=timestamps, 
                                                global_tuning=True, 
                                                evolution_tuning=True, 
                                                nr_bins=20)

topics_over_time

Topic                                             Words  Frequency  \
0        -1        donald, keychain, champion, trump, contest         20   
1         0        donald, execute, imagination, step, randal          9   
2         2   begun, schedule, ahead, international, scotland          1   
3         3         cling, wallflower, persona, walls, rather          1   
4        10            independence, safe, here, enjoy, happy          1   
...     ...                                               ...        ...   
1880    162                 ratings, fredo, frank, bad, based          2   
1881    163                        yes, no, way, absolutely,           2   
1882    164  twitter, trending, section, trends, conservative         13   
1883    165                    york, eaten, hell, new, blasio          4   
1884    167     mixing, courthouse, mocked, notes, prosecuted          3   

                         Timestamp  \
0    2009-04-30 12:30:07.596999936   
1    2009-04-30 12:30:07.596999936   
2    2009-04-30 12:30:07.596999936   
3    2009-04-30 12:30:07.596999936   
4    2009-04-30 12:30:07.596999936   
...                            ...   
1880 2020-06-09 07:29:57.849999872   
1881 2020-06-09 07:29:57.849999872   
1882 2020-06-09 07:29:57.849999872   
1883 2020-06-09 07:29:57.849999872   
1884 2020-06-09 07:29:57.849999872   

                                             Name  
0                                 -1_the_to_is_of  
1                    0_run_president_trump_donald  
2                    2_golf_course_doral_scotland  
3                3_border_wall_immigration_mexico  
4                    10_veterans_honor_heroes_our  
...                                           ...  
1880               162_ratings_machine_show_sided  
1881                    163_yes_no_absolutely_way  
1882  164_twitter_trending_conservative_sectio...  
1883                     165_york_ny_new_wonerful  
1884             167_jury_judge_guilty_foreperson  

[1885 rows x 5 columns]

<br>


## 可视化DTM

In [52]:
#模型中一共有169个主题，这里显示前Top10的主题的演变
topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=10)

## 公众号: 大邓和他的Python

[![](img/management_data_mining_with_python_course2.png)](https://hidadeng.github.io/blog/management_python_course/)
